## 1. Imports

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

from load_data import train_ds, val_ds, test_ds

In [ ]:
#determine the number of classes
for _, labels in train_ds:
    num_classes = len(labels[0])
    break

## 2. Create a base case that needs to be surpassed

In [ ]:
def random_classifier(dataset, num_classes):
    random_predictions = []
    true_labels = []
    
    for _, labels in dataset:
        batch_size = labels.shape[0]
        random_preds = np.random.randint(0, num_classes, size=batch_size)
        random_predictions.extend(random_preds)
        
        true_labels.extend(np.argmax(labels.numpy(), axis=1))
    
    return np.array(random_predictions), np.array(true_labels)

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate_random_classifier(random_predictions, true_labels):
    accuracy = accuracy_score(true_labels, random_predictions)
    print(f'Random Classifier Accuracy: {accuracy * 100:.2f}%')

In [ ]:
random_predictions, true_labels = random_classifier(train_ds, num_classes)
evaluate_random_classifier(random_predictions, true_labels)

## 3. Create a model

**I will use fine-tuning due to the small amount of computer power. Let's take the ConvNext model (Tiny version) as a basis, other high-quality options: Vit, swin, BEiT, EfficientNet. Then I will finish training the model on my data**

In [ ]:
conv_base = keras.applications.ConvNeXtTiny(
    model_name="convnext_tiny",
    include_top=False,
    weights='imagenet',
    input_shape = (224, 224, 3)
)

In [ ]:
x = layers.GlobalAveragePooling2D()(conv_base.output)
x = layers.Dense(256, activation='relu')(x)

outputs = layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs=conv_base.input, 
                    outputs=outputs, 
                    name='cards_classification')

#Freeze the layers of the base model
for layer in conv_base.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import AdamW

#Use categorical_crossentropy, because the data was encoded using categorical label mode
#The reason for using a low learning rate is the need to limit the amount of changes made
#to the representations of the three pre-trained layers. Making too many changes can damage these views.
model.compile(
    loss="categorical_crossentropy",
    optimizer=AdamW(learning_rate=1e-4),
    metrics=["accuracy"]
)

In [ ]:
from keras.callbacks import ModelCheckpoint

#use callbacks to save the model at the optimal point
callbacks = [
    ModelCheckpoint(
    filepath='cards_classification.keras',
    save_best_only=True,
    monitor='val_loss'
    )
]

## 4. Fit a model

In [ ]:
history = model.fit(
    train_ds,
    epochs=100,
    validation_data=val_ds,
    callbacks=callbacks
)

## 5. Visualization of learning

In [ ]:
import matplotlib.pyplot as plt

#Graph of losses at the training and validation stage
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, "bo", label="Losses at the training stage")
plt.plot(epochs, val_loss_values, "b", label="Losses at the validation stage")
plt.title("Losses at the training and validation stage")
plt.xlabel("Epochs")
plt.ylabel("Losses")

plt.legend()
plt.show()

In [ ]:
plt.clf() 

#Graph of accuracy at the training and validation stage
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Accuracy at the training stage")
plt.plot(epochs, val_acc, "b", label="Accuracy at the validation stage")
plt.title("Accuracy at the training and validation stage")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()